## Datasets. 3차원 공간 (i.e. 2 features)
## Regression? 
1. z = x + y
2. z = x * y
3. z = cos(x) + sin(y)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt

In [ ]:
x, y = np.meshgrid(np.linspace(0,1.5,130),np.linspace(0,1.5,130))

In [ ]:
features = torch.from_numpy(np.column_stack((x.ravel(), y.ravel())))
features = features.to(dtype=torch.float32)
features.shape

In [ ]:
z1 = features[:,0] + features[:,1]
z1, z1.shape

In [ ]:
z2 = (features[:,0]-0.5) * (features[:,1]-0.5)
z2.shape, z2

In [ ]:
z3 = (features[:,0]*10).cos() + (features[:,1]*10).sin()
z3, z3.shape

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self,
                    input_features: int,
                    output_features: int,
                    hidden_units: int = 8,
                    only_linear: bool = False) -> None:
        '''
        Initializes multi-class classification model.

        Args:
            input_features (int): Number of input features to the model
            output_features (int): Number of output features (Number of output classes)
            hidden_units (int): Number of hidden units between layers, default 8
            only_linear (bool): if ```True```, Not use Non-Linear activation Function

        Returns:
            None

        Example:
            ```
            model = BlobClassifier(input_features=3,
                                    ouput_features=4,
                                    hidden_units=8)
            ```
        '''
        super().__init__()

        # Layers
        if only_linear:
            self.linear_layer_stack = nn.Sequential(
                nn.Linear(in_features=input_features,
                            out_features=hidden_units),
                nn.Linear(in_features=hidden_units,
                            out_features=hidden_units),
                nn.Linear(in_features=hidden_units,
                            out_features=output_features)
            )
        else:
            self.linear_layer_stack = nn.Sequential(
                nn.Linear(in_features=input_features,
                            out_features=hidden_units),
                nn.ReLU(),
                nn.Linear(in_features=hidden_units,
                            out_features=hidden_units),
                nn.ReLU(),
                nn.Linear(in_features=hidden_units,
                            out_features=hidden_units),
                nn.ReLU(),
                nn.Linear(in_features=hidden_units,
                            out_features=output_features)
            )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [ ]:
class SimpleTransformer(nn.Module):
    def __init__(self,embed_dim):
        super().__init__()
        self.embed = nn.Linear(2,embed_dim) # b, 16

        self.qkv = nn.Linear(embed_dim, embed_dim*3)

        self.linear1 = nn.Linear(embed_dim, embed_dim)
        self.linear2 = nn.Linear(embed_dim, 1)

        self.ln = nn.LayerNorm([embed_dim])

        self.act = nn.PReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        # x,y -> 16 feature 
        x = self.embed(x)
        # 16 -> 16 * 3 (qkv) 16 16 16
        qkv = self.qkv(x).reshape(batch_size, 3, -1).permute(1,0,2).unsqueeze(-1) # batch 48a
        # 16 16 * 16 * 1
        q, k, v = qkv[0],qkv[1],qkv[2]

        attention = q @ k.transpose(1,2)

        attention = attention.softmax(dim=-1)

        x = attention @ v

        x = x.squeeze()


        
        x = self.linear1(x)
        x = self.act(x)


        x = self.linear2(x)
        
        return x

In [ ]:
class Multiple(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Linear(2,8)
        self.l1 = nn.Linear(8, 8)
        self.l2 = nn.Linear(8, 8)
        self.l3 = nn.Linear(8, 8)

        self.head = nn.Linear(8,1)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.embed(x)
        
        wx1 = self.act(self.l1(x)) + x

        x2 = wx1 * x
        wx2 = self.act(self.l2(x2)) + wx1

        x3 = wx2 * x
        wx3 = self.act(self.l2(x3)) + wx2

        out = self.head(wx3)

        return out

In [ ]:
torch.ones(32,16,16).flatten(1).shape

In [ ]:
from torchinfo import summary

model = Multiple()


summary(model,input_size=(32,2))

## Train

In [ ]:
import torch
import torch.nn as nn

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# model1 = SimpleTransformer(embed_dim=16).to(device)
# model2 = SimpleTransformer(embed_dim=16).to(device)
model1 = Multiple().to(device)
model2 = Multiple().to(device)
model3 = Multiple().to(device)

# model1 = SimpleMLP(input_features=2,
#                   output_features=1,
#                   hidden_units=64).to(device)
# model2 = SimpleMLP(input_features=2,
#                   output_features=1,
#                   hidden_units=64).to(device)
# model3 = SimpleMLP(input_features=2,
#                   output_features=1,
#                   hidden_units=64).to(device)


features = features.to(device)
z1, z2, z3 = z1.to(device), z2.to(device), z3.to(device)

loss_fn = nn.MSELoss()
optimizer1 = torch.optim.SGD(model1.parameters(), lr=1e-1)
optimizer2 = torch.optim.SGD(model2.parameters(), lr=1e-1)
optimizer3 = torch.optim.SGD(model3.parameters(), lr=1e-1)

In [ ]:
## Train Test Split
X_train = features[:11000]
z1_train = z1[:11000].unsqueeze(1)
z2_train = z2[:11000].unsqueeze(1)
z3_train = z3[:11000].unsqueeze(1)

X_test = features[11000:]
z1_test = z1[11000:].unsqueeze(1)
z2_test = z2[11000:].unsqueeze(1)
z3_test = z3[11000:].unsqueeze(1)

X_train.shape, z1_train.shape, z2_train.shape, z3_train.shape, X_test.shape, z1_test.shape, z2_test.shape, z3_test.shape


In [ ]:
def train_epoch(epoch,model, loss_fn, optimizer, X_train, X_test, y_train, y_test):
    model.train()
    logit = model(X_train)
    loss = loss_fn(logit, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()

    test_logit = model(X_test)
    test_loss = loss_fn(test_logit, y_test)
    if epoch % 100 ==0:
        print(f'Epoch:{epoch} | TestLoss: {test_loss} | TrainLoss: {loss}')

In [ ]:
# modelc.train()
# out = modelc(X_train)
# optimizer1.zero_grad()
# loss = loss_fn(out,z1_train)
# loss.backward()
# print(out, loss, modelc.embed.weight.grad)

# optimizer1.step()



In [17]:
epochs = 1000



print('Training Model1 | z = x + y')
for epoch in range(epochs):
    
    train_epoch(epoch=epoch,model=model1,
                loss_fn=loss_fn,
                optimizer=optimizer1,
                X_train=X_train,
                X_test=X_test,
                y_train=z1_train,
                y_test=z1_test)
    
print('Training Model2 | z = x * y')
for epoch in range(epochs):

    train_epoch(epoch=epoch,model=model2,
                loss_fn=loss_fn,
                optimizer=optimizer2,
                X_train=X_train,
                X_test=X_test,
                y_train=z2_train,
                y_test=z2_test)
    
print('Training Model1 | z = cos(x) + cos(y)')
for epoch in range(epochs):

    train_epoch(epoch=epoch,model=model3,
                loss_fn=loss_fn,
                optimizer=optimizer3,
                X_train=X_train,
                X_test=X_test,
                y_train=z3_train,
                y_test=z3_test)

Epoch:400 | TestLoss: 1.5420275926589966 | TrainLoss: 0.6862057447433472
Epoch:500 | TestLoss: 1.3323675394058228 | TrainLoss: 0.6645383238792419
Epoch:600 | TestLoss: 1.0952837467193604 | TrainLoss: 0.6278818845748901
Epoch:700 | TestLoss: 0.7604601383209229 | TrainLoss: 0.5863206386566162
Epoch:800 | TestLoss: 0.8408332467079163 | TrainLoss: 0.5620527863502502
Epoch:900 | TestLoss: 1.3907132148742676 | TrainLoss: 0.541201114654541


In [18]:
def plot3d(X, X_train, X_test, y, model, title):
    y_train = model(X_train)
    y_test = model(X_test)

    X, X_train, X_test = X.cpu().numpy(), X_train.cpu().numpy(), X_test.cpu().numpy()
    y, y_train, y_test = y.detach().cpu().numpy(), y_train.detach().cpu().numpy(), y_test.detach().cpu().numpy()


    fig = plt.figure(figsize=(24,24))
    ax = fig.add_subplot(projection='3d')


    ax.scatter(X[:,0],X[:,1],y, label='true')
    ax.scatter(X_train[:,0],X_train[:,1],y_train, label='train')
    ax.scatter(X_test[:,0],X_test[:,1],y_test, label='test')
    fig.legend(fontsize='20')
    plt.title(title, fontsize='30')
    plt.show()
    

In [19]:
plot3d(features, X_train, X_test, z1, model1,'z = x + y (wx + wx^2 + wx^3)| 2000 Epoch')

In [ ]:
plot3d(features, X_train, X_test, z2, model2,'z = x * y (wx + wx^2 + wx^3)| 4000 Epoch')

In [ ]:
plot3d(features, X_train, X_test, z3, model3,'z = cos(x) + sin(y) (wx + wx^2 + wx^3) | 4000 Epoch')